In [4]:
!nvidia-smi

Fri Mar 13 13:41:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 440.44       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:17:00.0 Off |                  N/A |
|  0%   53C    P2   172W / 300W |  10053MiB / 11019MiB |     74%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:65:00.0 Off |                  N/A |
| 44%   63C    P2   280W / 300W |  10889MiB / 11018MiB |     73%      Default |
+-------------------------------+----------------------+----------------------+
        

In [1]:
#!pip install iterative-stratification

In [2]:
import pandas as pd
import numpy as np

In [3]:
config = {
    'SEED': 32,
    'PATH_DATA': '/home/user-adm/ml/nikitam_bengali/BENGALI/',
    'N_FOLDS': 5,
}
config['VAL_SIZE'] = 1/config['N_FOLDS']

In [4]:
train_df = pd.read_csv(config['PATH_DATA']+'shorter_train.csv')
print(train_df.shape)
train_df.head()

(180756, 5)


,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_5,153,9,0,স্পো


In [5]:
train_df['id'] = train_df['image_id'].apply(lambda x: int(x.split('_')[1]))

useful_cols = ['id', 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic']
X, y = train_df[useful_cols].values[:,0], train_df[useful_cols].values[:, 1:]

train_df['fold'] = np.nan

# https://www.kaggle.com/yiheng/iterative-stratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
mskf = MultilabelStratifiedKFold(n_splits=config['N_FOLDS'], random_state=config['SEED'])
for i, (_, test_index) in enumerate(mskf.split(X, y)):
    train_df.loc[test_index, 'fold'] = i
    
train_df['fold'] = train_df['fold'].astype('int')

/home/user-adm/venv_benghali/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [8]:
train_df.drop('id', axis=1, inplace=True)

In [9]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme,fold
0,Train_0,15,9,5,ক্ট্রো,3
1,Train_1,159,0,0,হ,0
2,Train_2,22,3,5,খ্রী,1
3,Train_3,53,2,2,র্টি,4
4,Train_5,153,9,0,স্পো,2


In [10]:
train_df.fold.value_counts()

3    36152
4    36151
2    36151
1    36151
0    36151
Name: fold, dtype: int64

In [11]:
train_df.to_csv('shorter_train_folds.csv', index=False, encoding='utf-8')